In [24]:
import pandas as pd
import numpy as np
from konlpy.tag import Hannanum
import gensim
import re

In [25]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from pprint import pprint

In [2]:
hannanum = Hannanum()

In [10]:
df = pd.read_csv("filelink.csv", encoding='utf-8',header=None)

In [11]:
df.columns = ['user_id', 'timestamp', 'title', 'contents']

In [12]:
df.tail(20)

,user_id,timestamp,title,contents
5630,4564440692,2017.04.05 08시 13분,(그마) 발리라 빠대에선 사기인거 같음. 진짜,근데 영리에선 그냥 후픽으로 뽑으면 쓸만한 정도인거 같은데그마급 유저가 빠대에서 잡...
5631,4564497238,2017.04.14 03시 13분,난투 매칭좀 바꿔라 쫌,아니 트롤만 계속 엮어주면 뭐 하자는거야
5632,4564507293,2017.04.13 15시 49분,발리라 만든 새기 머리 어떻게 된거 아니냐,"나왔을때부터 지금까지 저딴 케릭 만든새기 머리통 뒤에서 쌔게 한대 치고싶다,빡빡이의..."
5633,4564488123,2017.04.13 16시 02분,탈주 패널티 누적에 관하여...,저는 사실 탈주를 여러번 했습니다... 죄송합니다 죄인입니다. 그런데 탈주 많이해서...
5634,4564497137,2017.04.13 13시 40분,모든 게임에는 트롤촌이란 시스템이 있어야 한다,게임내에 고의트롤로 인해 고통받는 일반유저게임을 스트레스 풀고 즐기기 위해 하는것인...
5635,4564497108,2017.04.13 11시 53분,발리라 너프 안함?,까닥이나 매칭할때마다 상대팀 발리라만 보이고 울팀은 하나도 안잡히는데솔직히 특성이나...
5636,4564469356,2017.04.09 00시 41분,카시아 지금 황금밸런스인거같아요,"사거리 1만 더 길었어도 어떤 괴물이 나올지 상상이 안가네,실명에 눈알을 탁 치고 ..."
5637,4564496638,2017.04.12 12시 53분,히오스2.0,히오스2.0에서 퀘스트 달성해서 얻은 잼은 본섭 적용되나요? ...
5638,4564487374,2017.04.13 00시 42분,영웅리그 듀오,영웅리그 듀오 가능하게 다시 바뀌어야 합니다.팀리그는 10분 가까히 기다려야 잡히고...
5639,4564486851,2017.04.12 16시 40분,머가리에 똥만 들어있는 블리자드,니네는 뇌가 있냐? 새로운 유저나 부케 파는 사람들 팀으로 빠대해서 mmr 올리고 ...


In [15]:
df[df['contents'].isnull()]

,user_id,timestamp,title,contents
899,4566549755,NaN,새로운 영웅 '문재인' 제안합니다,NaN


In [38]:
df_by_user = df[df['contents']=='1']
df_by_user['timestamp'].tolist()


# 전처리 되도록이면 판다스로 하기. 유저 아이디 중복을 찾은다음에. 시간 차이가 몇 안나는 것들은 빼기(몇 분 이상 차이나면 없애는 방향으로)

['2017.12.07 12시 05분',
 '2017.10.10 12시 58분',
 '2017.10.02 14시 56분',
 '2017.05.17 13시 19분',
 '2017.04.11 12시 21분',
 '2017.04.11 12시 20분']

In [31]:
user_list = df['user_id'].unique()
for uid in user_list[:10]:
    print(df[df['user_id']==uid])

      user_id           timestamp                title  \
0  4566886652  2018.08.28 05시 22분  등급전 MMR초기화 잘 생각해보세요   

                                            contents  
0  시즌 좀 쉬었다고 등급전을 이렇게 힘들게 올라가야합니까?실제실력이 몇계단이나 차이나...  
      user_id           timestamp                                    title  \
1  4566866650  2018.08.28 04시 09분  진짜 이 빌어먹을 공산당 게임 갑자기 경험치 기여도 높은분들 우대해주면   

                                            contents  
1  무슨일 생길지 개인적으로 궁금하다...현재까지 mmr시스템을 그냥 재미삼아라도 (실...  
      user_id           timestamp                          title  \
2  4566886643  2018.08.28 01시 20분  영웅 리그에 상대편으로 매칭 희망 기능이 필요합니다.   

                                            contents  
2  영웅 리그에 상대편으로 매칭 희망 기능이 필요합니다.오버워치에도 해당 기능이 있다가...  
      user_id           timestamp              title  \
3  4565446647  2017.12.02 17시 28분  이럴거면 그냥 켈투자드 삭제해라   

                                            contents  
3  켈투자드 출시할때 기대 정말 많이 했다맵을 만든 것도 아니고 영웅 하나 출시하는데 ...  
      user_id           timestamp   

In [28]:
import datetime
ex_date = df['timestamp'][0]
datetime.datetime()

TypeError: Required argument 'month' (pos 2) not found

In [36]:
for i in range(len(df)):
    df['contents'][i] = str(df['contents'][i])

/Users/Yeoni/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [56]:
for i in range(3):
    hannanum.nouns(df["contents"][i])

In [57]:
df.head()

,user_id,timestamp,title,contents
0,4566886652,2018.08.28 05시 22분,등급전 MMR초기화 잘 생각해보세요,시즌 좀 쉬었다고 등급전을 이렇게 힘들게 올라가야합니까?실제실력이 몇계단이나 차이나...
1,4566866650,2018.08.28 04시 09분,진짜 이 빌어먹을 공산당 게임 갑자기 경험치 기여도 높은분들 우대해주면,무슨일 생길지 개인적으로 궁금하다...현재까지 mmr시스템을 그냥 재미삼아라도 (실...
2,4566886643,2018.08.28 01시 20분,영웅 리그에 상대편으로 매칭 희망 기능이 필요합니다.,영웅 리그에 상대편으로 매칭 희망 기능이 필요합니다.오버워치에도 해당 기능이 있다가...
3,4565446647,2017.12.02 17시 28분,이럴거면 그냥 켈투자드 삭제해라,켈투자드 출시할때 기대 정말 많이 했다맵을 만든 것도 아니고 영웅 하나 출시하는데 ...
4,4566857165,2018.08.26 17시 41분,모델링관련하여,캐릭터모델링 개편 부탁드립니다. 원작의 위엄이 사라진 다리짧은 아서스 뚱뚱해진 제이...


In [37]:
df.shape

(5650, 4)

In [91]:
tokens = []

for i in list(df["contents"]):
    k = hannanum.nouns(i)
    tokens.append(k)

In [71]:
type(tokens)

list

In [104]:
tokens_df = pd.DataFrame({"tokens":tokens})

In [105]:
tokens_df.head()

,tokens
0,"[인증서버, 관리, 신입, ,이래놓고, 보상, 어영부영, 넘어갈려고하, 문제, 있는..."
1,"[픽밴, 인터넷, 문제, 거, 탈주자, 처리하는건, 너무한거, 픽밴, 인터넷, 10..."
2,"[시즌, 등급전, 실제실력, 몇계단, 차, 유저, 기록, 배치, 진행한다면그, 유저..."
3,"[무슨일, 개, 적, 궁금, 현재, 시스템, 재미삼아라, 실질적, 승리, 알고리즘,..."
4,"[영웅, 리그, 상대편, 매칭, 희망, 기능, 필요, 오버워치, 해당, 기능, 매칭..."


In [106]:
df["tokens_contents"] = tokens

In [108]:
id2word = corpora.Dictionary(df["tokens_contents"])

In [109]:
texts = df["tokens_contents"]

In [111]:
corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 3), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1)]]


In [113]:
id2word[0]

'시간'

In [114]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('시간', 1),
  ('상대방', 1),
  ('어영부영', 1),
  ('유저', 1),
  ('게임', 3),
  ('문제', 1),
  ('것도아니', 1),
  ('서버', 2),
  ('사소', 1),
  ('@#', 1),
  ('이상', 1),
  ('뒤', 1),
  (',이래놓고', 1),
  ('해', 1),
  ('사태', 1),
  ('구매', 1),
  ('튕긴다음', 1),
  ('소비자', 1),
  ('솔찍히', 1),
  ('처', 1),
  ('관리', 1),
  ('점검', 1),
  ('자기', 1),
  ('수정', 1),
  ('해야지아무말', 1),
  ('인증서버', 1),
  ('사용', 1),
  ('넘어갈려고하', 1),
  ('공짜', 1),
  ('것들', 1),
  ('40분', 1),
  ('적어놓는거', 1),
  ('보상', 1),
  ('신입', 1),
  ('불나드', 1),
  ('우리', 1),
  ('있는거', 1),
  ('맘이고긴급한', 1)]]

In [115]:
# Build LDA model

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=20,
                                            random_state=100, # Seed
                                            update_every=1, # 청크사이즈 마다 모델 업데이트 (메모리 관리 패러미터)
                                            chunksize=100,# 한 번에 고려할 문서의 수
                                            passes=10, # 알고리즘이 전체 코퍼스를 통과해야 하는 횟수
                                            alpha='auto',
                                            per_word_topics=True) # 주어진 한 단어가 가장 가능성 있는 주제를 추출하도록 허용

In [118]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.050*"보상" + 0.026*"영웅리그" + 0.018*"등급전" + 0.017*"배치" + 0.014*"팀리그" + '
  '0.010*"표시" + 0.010*"시즌" + 0.010*"해결" + 0.009*"등급" + 0.008*"친구"'),
 (1,
  '0.027*"5월" + 0.024*"완료" + 0.021*"서버" + 0.019*"진행" + 0.016*"1주차" + '
  '0.016*"점검" + 0.015*"2주차" + 0.013*"이용" + 0.011*"시간" + 0.011*"서비스"'),
 (2,
  '0.032*"이거" + 0.030*"디바" + 0.027*"기존" + 0.016*"정도" + 0.014*"인정" + 0.011*"옵치" '
  '+ 0.010*"조건" + 0.010*"니들" + 0.010*"ㅋㅋ" + 0.008*"범위"'),
 (3,
  '0.037*"은신" + 0.013*"제라툴" + 0.012*"도살자" + 0.011*"ㅋ" + 0.009*"레오릭" + '
  '0.007*"안주" + 0.006*"버그" + 0.005*"학습효과" + 0.005*"강제" + 0.005*"따"'),
 (4,
  '0.028*"님" + 0.011*"화물" + 0.010*"ㅠㅠ" + 0.010*"공개" + 0.009*"댓글" + 0.009*"트롤들" '
  '+ 0.007*"핵" + 0.007*"탑" + 0.007*"." + 0.006*"진입"'),
 (5,
  '0.060*"것" + 0.047*"게임" + 0.037*"수" + 0.031*"히오스" + 0.019*"저" + 0.018*"생각" + '
  '0.015*"이" + 0.014*"때" + 0.014*"유저" + 0.014*"진짜"'),
 (6,
  '0.053*"스킨" + 0.029*"오니겐지" + 0.014*"상자" + 0.012*"초상화" + 0.011*"전리품" + '
  '0.010*"상품" + 0.010*"스프레이" + 0.010*"누더기" + 0.009*"묶음

In [ ]:
# 방식이 여러가지로 구성할 수 있음

In [120]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.
 
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=df["tokens_contents"], dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -11.300457911573966

Coherence Score:  0.4453570196114264


In [125]:
import pyLDAvis
from pyLDAvis import gensim

In [126]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      30.157355        1       1 -0.462492  0.001050
13      6.390772        1       2 -0.047572  0.060262
7       5.912201        1       3 -0.039180  0.082990
10      5.327456        1       4  0.021797  0.026785
2       4.372230        1       5  0.039128 -0.014169
9       4.302891        1       6 -0.020089 -0.305693
15      4.059350        1       7  0.015796  0.069840
0       4.046431        1       8  0.017780  0.056694
19      3.901637        1       9  0.026108 -0.114614
11      3.857598        1      10  0.036885 -0.026876
8       3.295584        1      11  0.047440  0.025739
18      3.223605        1      12  0.043912 -0.002759
1       3.079646        1      13 -0.035601  0.065788
6       2.809660        1      14  0.031011  0.028499
17      2.739616        1      15  0.049514  0.025303
12      2.688790        1      16  0.059040  0.014527
16      2.642550        1      17  0.041133 -0.002345
3       2.536460        1      18  0.053685 -0.013994
4       2.406784        1      19  0.060271  0.008469
14      2.249384        1      20  0.061435  0.014502, topic_info=      Category         Freq  Term        Total  loglift  logprob
term                                                            
58351  Default  4619.000000     것  4619.000000  30.0000  30.0000
39505  Default  3636.000000    게임  3636.000000  29.0000  29.0000
32497  Default  2874.000000     수  2874.000000  28.0000  28.0000
26316  Default   841.000000   이벤트   841.000000  27.0000  27.0000
32966  Default  2367.000000   히오스  2367.000000  26.0000  26.0000
16095  Default   896.000000    매칭   896.000000  25.0000  25.0000
35265  Default   511.000000    보상   511.000000  24.0000  24.0000
55723  Default  1371.000000    생각  1371.000000  23.0000  23.0000
50531  Default  1485.000000     저  1485.000000  22.0000  22.0000
42571  Default   381.000000    스킨   381.000000  21.0000  21.0000
6268   Default  1160.000000     이  1160.000000  20.0000  20.0000
40494  Default   592.000000    빠대   592.000000  19.0000  19.0000
40097  Default  1084.000000    유저  1084.000000  18.0000  18.0000
44484  Default   393.000000    탈주   393.000000  17.0000  17.0000
35563  Default   429.000000   작성자   429.000000  16.0000  16.0000
51739  Default  1192.000000    진짜  1192.000000  15.0000  15.0000
12056  Default  1136.000000     때  1136.000000  14.0000  14.0000
24485  Default   399.000000    시공   399.000000  13.0000  13.0000
52801  Default   621.000000     적   621.000000  12.0000  12.0000
992    Default   360.000000    이거   360.000000  11.0000  11.0000
29288  Default   712.000000  블리자드   712.000000  10.0000  10.0000
44706  Default   388.000000    특성   388.000000   9.0000   9.0000
9343   Default   948.000000   유저들   948.000000   8.0000   8.0000
5571   Default   312.000000    겐지   312.000000   7.0000   7.0000
26954  Default   337.000000    디바   337.000000   6.0000   6.0000
7060   Default   358.000000     욕   358.000000   5.0000   5.0000
48449  Default   427.000000     애   427.000000   4.0000   4.0000
4799   Default   452.000000    시간   452.000000   3.0000   3.0000
22007  Default   964.000000    사람   964.000000   2.0000   2.0000
6885   Default   833.000000     말   833.000000   1.0000   1.0000
...        ...          ...   ...          ...      ...      ...
48609  Topic20    58.403146    어이    59.241355   3.7803  -4.5861
20231  Topic20    45.976812    인공    46.815021   3.7764  -4.8254
5415   Topic20    41.873896    인증    42.712105   3.7747  -4.9188
2580   Topic20    39.249222    중간    40.087431   3.7734  -4.9836
42071  Topic20    36.130240    상점    36.968449   3.7716  -5.0664
59004  Topic20    32.803915     부    33.642124   3.7693  -5.1629
23216  Topic20    30.148696    정예    30.986905   3.7671  -5.2474
2849   Topic20    30.020824    방치    30.859033   3.7670  -5.2516
1152   Topic20    28.134895     채    28.973104   3.7652  -5.3165
46981  Topic20    27.814563  많이해서    28

In [ ]:
r'[ㄱ-ㅎㅏ-ㅣ]+'
#한글 정규표현식 찾아보기